In [1]:
import read_mta_stations as s
import read_mta_turnstile as t
import read_tech_compaines as c
import create_station_heatmap as h

In [2]:
stations = s.read_mta_stations()
stations.sample(5)

,key,name,LINE,lon,lat
216,217,Junius St,3-4,-73.90244864183562,40.66358900181724
131,132,Newkirk Ave,2-5,-73.94829990822407,40.63999124275311
285,286,Dyckman St,1,-73.92553600006474,40.86053100138796
6,7,Cathedral Pkwy (110th St),A-B-C,-73.95806670661364,40.800581558114956
394,395,Spring St,A-C-E,-74.00373899843763,40.72622700129312


In [3]:
companies = c.read_tech_companies()
companies

,COMPANY,LAT,LON
0,Accenture Content Studio,40.725410,-74.004690
1,Accenture,40.763000,-73.979030
2,Thoughtworks,40.744678,-73.984901
3,Amazon,40.753050,-73.999440
4,Bank of America,40.755390,-73.984710
5,Facebook,40.730701,-73.991531
6,HP,40.744830,-73.999460
7,Microsoft,40.756989,-73.989708
8,Two Sigma,40.723270,-74.004390
9,Workday,40.748170,-73.985000


In [ ]:
tur